In [1]:
from collections import defaultdict
import pickle
from datetime import datetime, timedelta

import pandas as pd

### Chicago
Data source for Chicago:

https://www.chicago.gov/city/en/sites/covid-19/home/latest-data.html

Create a dict of the form {days from Feb 29: number of deaths}, and save it as a pandas DataFrame starting with Feb 1 to the end of the dict.

In [52]:
chi_death_dict = defaultdict(int, {16: 1, 17: 1, 18: 1, 19: 2, 21: 2,
                                   22: 2, 23: 2, 24: 2, 25: 5, 26: 3,
                                   27: 8, 28: 4, 29: 21, 30: 12,
                                   31: 15, 32: 24, 33: 12, 34: 24,
                                   35: 20, 36: 28, 37: 21, 38: 24,
                                   39: 34, 40: 25, 41: 31, 42: 29,
                                   43: 30, 44: 39, 45: 31, 46: 30,
                                   47: 28, 48: 30, 49: 35, 50: 33,
                                   51: 28, 52: 20, 53: 13})

In [53]:
chi_last_day = max(chi_death_dict.keys())

In [54]:
chi_index = [datetime(2020, 2, 1) + timedelta(days=day) for day in range(29 + chi_last_day)]
chi_deaths =[chi_death_dict[day - 28] for day in range(29 + chi_last_day)]

In [55]:
chi_death_df = pd.DataFrame(chi_deaths, index=chi_index, columns=['deaths'])

In [56]:
# Inspect that last few days
chi_death_df.tail()

,deaths
2020-04-18,35
2020-04-19,33
2020-04-20,28
2020-04-21,20
2020-04-22,13


In [57]:
chi_death_df.to_pickle('../data/chi_deaths.pickle')

### Illinois
Data source for Illinois:

http://www.dph.illinois.gov/covid19/covid19-statistics

`il_cum_death_list` is the list of cumulative deaths on each day, starting on March 16. Again, save it as a pandas DataFrame starting with Feb 1 to the end of the dict.

In [58]:
il_cum_death_list = [0, 1, 3, 4, 5, 6, 9, 12, 16, 19, 26, 34, 47,
                     65, 72, 99, 141, 157, 210, 243, 274, 307, 380,
                     462, 528, 596, 677, 720, 794, 868, 948, 1072,
                     1134, 1259, 1290, 1349, 1468, 1565, 1688,]

In [59]:
# New deaths from March 17
il_death_list = [now - prev 
                 for now, prev in zip(il_cum_death_list[1:],
                                      il_cum_death_list[:-1])]

In [60]:
il_death_dict = defaultdict(int, {index + 17: diff
                                  for index, diff in enumerate(il_death_list)})

In [61]:
il_num_days = len(il_death_list)

In [62]:
num_days = len(il_death_list)

In [63]:
il_index = [datetime(2020, 2, 1) + timedelta(days=day) for day in range(29 + 16 + num_days)]
il_deaths =[il_death_dict[day - 28] for day in range(29 + 16 + num_days)]

In [64]:
il_death_df = pd.DataFrame(il_deaths, index=il_index, columns=['deaths'])

In [65]:
# Inspect that last few days
il_death_df.tail()

,deaths
2020-04-19,31
2020-04-20,59
2020-04-21,119
2020-04-22,97
2020-04-23,123


In [66]:
il_death_df.to_pickle('../data/il_deaths.pickle')